<a href="https://colab.research.google.com/github/min02yam/project_2022_03/blob/main/sklearn_1215.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings(action='ignore') 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/2022_03/data/train10_0.csv")
test = pd.read_csv("/content/drive/MyDrive/2022_03/data/test10_0.csv")
submission = pd.read_csv("/content/drive/MyDrive/2022_03/data/sample_submission.csv")

feature = train.drop(['predicted_weight_g'],axis=1).columns
target = 'predicted_weight_g'

In [3]:
import random
import os
import numpy as np
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
set_seed(49)

In [69]:
list(np.arrange(0.9, 0.1))

TypeError: ignored

In [81]:
np.arange(1e-1,1e-3,9e-2)

array([], dtype=float64)

In [85]:
list(np.arange(1e-4,1e-3,1e-4))

0.001, 0.01, 0.05, 0.1


     {'preprocess':[None],
      'regressor': [XGBRegressor()],
      'regressor__objective' : ['reg:squarederror'],
      'regressor__n_estimators': list(range(100, 3600,100)),
      'regressor__max_depth': list(range(2,10)),
      'regressor__eta' :  list(np.arange(1e-4,1e-3,1e-4)),
      'regressor__subsample' : [0.5,0.6,0.7,0.8]},


           #{'preprocess':[None],
     # 'regressor': [XGBRegressor()],
     # 'regressor__objective' : ['reg:squarederror'],
     # 'regressor__n_estimators': list(range(100, 3600,100)),
     # 'regressor__max_depth': list(range(2,10)),
     # 'regressor__eta' :  list(np.arange(1e-4,1e-3,1e-4)),
     # 'regressor__subsample' : [0.5,0.6,0.7,0.8]},

[0.0001,
 0.0002,
 0.00030000000000000003,
 0.0004,
 0.0005,
 0.0006000000000000001,
 0.0007000000000000001,
 0.0008,
 0.0009000000000000001]

In [114]:
list(range(500, 2500,500))

[500, 1000, 1500, 2000]

In [127]:
# test
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV 

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, RANSACRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor,GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor

pip = Pipeline((("preprocess", MinMaxScaler()), ("regressor", LinearRegression()))) 
grid_parm = [
    
    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [LinearRegression()]},
     
     {'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [Ridge()],
      'regressor__alpha': [0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]},
     
     {'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [Lasso()]},

    #{'preprocess': [StandardScaler(), MinMaxScaler()],
    # 'regressor': [ElasticNet()],
    # 'regressor__alpha' : np.arange(1e-4,1e-3,1e-4).tolist(),
    # 'regressor__l1_ratio': np.arange(0.1,1.0,0.1).tolist(),
    # 'regressor__max_iter':[100000]

    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [KNeighborsRegressor()],
     'regressor__n_neighbors': list(range(1,25,2))},
     
     {'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [SVR()],
     'regressor__kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
     
     {'preprocess':[None],
      'regressor': [DecisionTreeRegressor()],
      'regressor__max_depth': list(range(2,15)),
      'regressor__max_leaf_nodes': [5, 50, 500, 5000]},
    
     {'preprocess':[None],
      'regressor': [RandomForestRegressor()],
      'regressor__n_estimators': [50,100,150]},
      
      {'preprocess':[None],
      'regressor': [AdaBoostRegressor()],
      'regressor__n_estimators': [50,100,150]},     

      {'preprocess':[None],
      'regressor': [GradientBoostingRegressor()],
      'regressor__n_estimators': [50,100,150]},
      
      {'preprocess':[None],
       'regressor': [XGBRegressor()]},

     {'preprocess':[None],
      'regressor': [LGBMRegressor()]},

     {'preprocess':[StandardScaler(), MinMaxScaler()],
      'regressor': [MLPRegressor()],
      'regressor__hidden_layer_sizes': [(100,),(100,100),(100,100,100)],  # 세번째, 히든레이어 세개, 노드개수 100개
      'regressor__max_iter' :[1000]}      

]

from sklearn.inspection import permutation_importance # sklearn 22 버전부터 해당
from sklearn.metrics import mean_squared_error, make_scorer
def root_mean_squared_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return mean_squared_error(y_test, y_pred,squared=False )

grid_m = GridSearchCV(pip, 
                      grid_parm, 
                      cv = 5,
                      return_train_score=True, 
                      verbose=3,
                      scoring = root_mean_squared_error)

grid_m.fit(train[feature], train[target])

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV 1/5] END preprocess=StandardScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) total time=   0.0s
[CV 2/5] END preprocess=StandardScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) total time=   0.0s
[CV 3/5] END preprocess=StandardScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) total time=   0.0s
[CV 4/5] END preprocess=StandardScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) total time=   0.0s
[CV 5/5] END preprocess=StandardScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) total time=   0.0s
[CV 1/5] END preprocess=MinMaxScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) total time=   0.0s
[CV 2/5] END preprocess=MinMaxScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) total time=   0.0s
[CV 3/5] END preprocess=MinMaxScaler(), regressor=LinearRegression();, score=(train=nan, test=nan) to

KeyboardInterrupt: ignored

In [123]:
grid_m.best_estimator_

Pipeline(steps=[('preprocess', MinMaxScaler()),
                ('regressor',
                 MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=1000))])

In [124]:
best_medel = grid_m.best_estimator_
pred = best_medel.predict(test[feature])

submission['predicted_weight_g'] = pred
submission['case'] = test['case']

submission.to_csv('mlp_10_0.csv', index=False) #
submission.head()

,DAT,predicted_weight_g,case
0,1,3.253650,0.0
1,2,2.968853,0.0
2,3,4.391424,0.0
3,4,5.080574,0.0
4,5,4.495380,0.0


In [126]:
from sklearn.inspection import permutation_importance # sklearn 22 버전부터 해당
from sklearn.metrics import mean_squared_error, make_scorer

# RMSE
def root_mean_squared_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return mean_squared_error(y_test, y_pred,squared=False )



# fix_model : 훈련된 모델
# X_train : 훈련데이터 Feature
# y_train : 훈련데이터 Target
# scoring : Feature 를 Shuffler 한 뒤, 예측값과 실제값을 어떤 Metric 을 사용해 비교할지
# n_repeats : 특정 Feature 를 몇번 Shuffle 할 것인지
# random_state : 난수 고정
result = permutation_importance(best_medel, train[feature], train[target], scoring = make_scorer(root_mean_squared_error,greater_is_better=False),
                            n_repeats=30)
# Feature label
Feature = train[feature] 

# Feature 중요도를 오름차순으로 정렬한 뒤, 해당 Feature 의 index 를 저장
sorted_result = result.importances_mean.argsort()

# 결과를 DataFrame 화
importances = pd.DataFrame(result.importances_mean[sorted_result], index=Feature.columns[sorted_result]).sort_values(0, ascending=False)   
importances

,0
DAT,36.315215
case,9.049442
시간당청색광량,8.371577
시간당분무량,7.343181
일간누적적색광량,6.702268
co2관측치,6.104581
일간누적청색광량,5.764596
시간당백색광량,5.328502
내부습도관측치,5.241828
일간누적백색광량,3.978451


In [ ]:

 0.2, 0.9, 0.1

Params: 
		tweedie_variance_power: 1.1
		max_depth : list(range(2,10))
		n_estimators: list(range(50, 4000,50))
		eta: list(np.arange(1e-4,1e-3,1e-4))
		subsample: [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    
    
    
    reg_lambda 


		colsample_bytree: 0.30000000000000004
		colsample_bylevel: 0.8
		min_child_weight: 0.05117429543357575
		reg_lambda: 1426.7293533924328

		reg_alpha: 7.346031097589848
		gamma: 8.062662439327799


            n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            max_depth=5, ## 개별 나무의 최대 깊이
            gamma = 0, ## gamma
            reg_lambda = 1, ## tuning parameter of l2 penalty
            random_state=100
        )



In [99]:
grid_m.best_params_

{'preprocess': MinMaxScaler(),
 'regressor': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=1000),
 'regressor__hidden_layer_sizes': (100, 100),
 'regressor__max_iter': 1000}

In [101]:
best_medel = grid_m.best_estimator_

In [92]:
grid_m.best_index_

122

In [ ]:
estimators = [('ridge', RidgeCV()),
              ('lasso', LassoCV(random_state=42)),
               ('knr', KNeighborsRegressor(n_neighbors=20,
                                           metric='euclidean'))]


final_estimator = GradientBoostingRegressor(
     n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1,
     random_state=42)

reg = StackingRegressor(
     estimators=estimators,
     final_estimator=final_estimator)

In [139]:
param_grid 

{'alpha': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
        0.0009]),
 'l1_ratio': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
 'max_iter': [100000]}

In [27]:


param_grid = {'alpha': np.arange(1e-4,1e-3,1e-4),
              'l1_ratio': np.arange(0.1,1.0,0.1),
              'max_iter':[100000]}

lasso_alph_range = np.arange(1e-4,1e-3,4e-5)
lasso_param_grid = {'alpha': lasso_alph_range}

     'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [ElasticNet()],

 
 
       'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [Ridge()],
     'regressor__alpha': [0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75], 
  
  
      'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [RidgeCV()],
     'regressor__alphas': [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75],
     
  
  
     'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [LassoCV()],
     'regressor__alphas': alphas2
          'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [Lasso()],
     'regressor__alpha': [1, 0.1, 0.001, 0.0005],

In [13]:
h_params=[{'n_estimators': 1833, 'max_depth': 5, 'learning_rate': 0.1121792759, 'verbose': 0}, 
          {'n_estimators': 9699, 'max_depth': 7, 'learning_rate': 0.1603481692, 'num_leaves': 10, 'verbose': -1}, 
          {'n_estimators': 7482, 'max_depth': 16, 'learning_rate': 0.08874432474, 'colsample_bylevel': 0.8985728179926714, 'subsample': 0.9456095953392613, 'min_child_samples': 76, 'max_bin': 441, 'l2_leaf_reg': 4.369161316123442}, 
          {'n_estimators': 6356, 'max_depth': 8, 'learning_rate': 0.05564907504, 'l2_leaf_reg': 4.861011004218151, 'min_child_samples': 23}, 
          {'n_estimators': 3797, 'max_depth': 11, 'learning_rate': 0.02776310885, 'l2_leaf_reg': 5.149047265858214, 'min_child_samples': 12}, 
          {'n_estimators': 3315, 'max_depth': 11, 'learning_rate': 0.07331924703}, 
          {'n_estimators': 8976, 'max_depth': 10, 'learning_rate': 0.08747145625, 'num_leaves': 10, 'min_child_weight': 74, 'alpha': 0.09206520231143506, 'subsample': 0.5296666607945134}, 
          {'n_estimators': 1048, 'max_depth': 20, 'learning_rate': 0.09961398955, 'num_leaves': 32}, 
          {'n_estimators': 13037, 'max_depth': 14, 'learning_rate': 0.08952616823, 'num_leaves': 16, 'verbose': -1}, 
          {'n_estimators': 12935, 'max_depth': 17, 'learning_rate': 0.06636082526, 'num_leaves': 17, 'verbose': -1}, 
          {'n_estimators': 5624, 'max_depth': 20, 'learning_rate': 0.01863652732, 'num_leaves': 164}, 
          {'n_estimators': 2473, 'max_depth': 19, 'learning_rate': 0.03943885644, 'num_leaves': 102}, 
          {'n_estimators': 6690, 'max_depth': 17, 'learning_rate': 0.01807948951, 'num_leaves': 119}, 
          {'n_estimators': 7093, 'max_depth': 18, 'learning_rate': 0.02448476821, 'num_leaves': 85}, 
          {'n_estimators': 4292, 'max_depth': 16, 'learning_rate': 0.03043742343, 'num_leaves': 97}, 
          {'n_estimators': 4168, 'max_depth': 13, 'learning_rate': 0.04860827553, 'num_leaves': 85}, 
          {'n_estimators': 2576, 'max_depth': 29, 'learning_rate': 0.0431085983, 'num_leaves': 245, 'verbosity': -1}, 
          {'n_estimators': 1593, 'max_depth': 42, 'learning_rate': 0.01033448085, 'num_leaves': 245, 'verbosity': -1}, 
          {'n_estimators': 836, 'max_depth': 42, 'learning_rate': 0.1540753462, 'num_leaves': 113, 'verbosity': -1}, 
          {'n_estimators': 727, 'max_depth': 34, 'learning_rate': 0.1600727117, 'num_leaves': 74, 'verbosity': -1}, 
          {'n_estimators': 9789, 'max_depth': 29, 'learning_rate': 0.06871880461, 'num_leaves': 46, 'verbosity': -1}, 
          {'n_estimators': 12266, 'max_depth': 20, 'learning_rate': 0.05269683494, 'num_leaves': 56, 'verbosity': -1}, 
          {'n_estimators': 3630, 'max_depth': 33, 'learning_rate': 0.1021564607, 'num_leaves': 70, 'verbosity': -1},
          {'n_estimators': 866, 'max_depth': 19, 'learning_rate': 0.05244185073, 'num_leaves': 20},
          {'n_estimators': 3725, 'max_depth': 16, 'learning_rate': 0.1291892976, 'num_leaves': 14}, 
          {'n_estimators': 11963, 'max_depth': 28, 'learning_rate': 0.06079772926, 'subsample': 0.9873435671739007, 'num_leaves': 18, 'verbose': -1, 'min_child_samples': 52, 'reg_lambda': 3.6402158460803955}, 
          {'n_estimators': 1185, 'max_depth': 18, 'learning_rate': 0.1782109397, 'num_leaves': 50, 'verbose': -1}]

#################################################################################

In [148]:
grid_m.best_params_

{'preprocess': MinMaxScaler(),
 'regressor': ElasticNet(alpha=0.1),
 'regressor__alpha': 0.1}

In [64]:
grid_m.score

<bound method BaseSearchCV.score of GridSearchCV(cv=5,
             estimator=Pipeline(steps=(('preprocess', MinMaxScaler()),
                                       ('regressor', LinearRegression()))),
             param_grid=[{'preprocess': [StandardScaler(), MinMaxScaler()],
                          'regressor': [Ridge(alpha=30)],
                          'regressor__alpha': [0.1, 0.3, 1, 3, 5, 10, 15, 30,
                                               50, 75]}],
             return_train_score=True, verbose=3)>

In [65]:
grid_m.best_score_

0.4269265847965662

In [12]:
grid_m.cv_results_

{'mean_fit_time': array([0.01394224, 0.01500349, 0.01289716, 0.00848818, 0.00568833,
        0.00713129, 0.00690007, 0.0080739 , 0.00616717, 0.00584145,
        0.00615368, 0.00604978, 0.00573335, 0.00778451, 0.0095962 ,
        0.00562077, 0.00564194, 0.00661478, 0.00538712, 0.00587792,
        0.00758615, 0.00749202, 0.01115031, 0.01369796]),
 'std_fit_time': array([6.53465495e-04, 1.67643437e-03, 5.43182187e-03, 3.62058855e-03,
        1.18405359e-04, 2.62875292e-03, 1.05859585e-03, 4.91040456e-03,
        4.79429186e-04, 5.11754025e-05, 4.31381603e-04, 1.02383678e-04,
        3.20131174e-04, 3.77967658e-03, 6.20841354e-03, 2.75327584e-04,
        2.74261712e-04, 2.17938912e-03, 1.54891389e-04, 4.46691099e-04,
        2.20741891e-03, 3.23379634e-03, 3.85165782e-03, 1.18048451e-03]),
 'mean_score_time': array([0.00262294, 0.0051723 , 0.00359015, 0.00236039, 0.00241847,
        0.0023551 , 0.00251398, 0.00247998, 0.00250983, 0.00235019,
        0.00246191, 0.00248857, 0.00245185, 0.00

In [13]:
grid_m.best_params_

{'preprocess': MinMaxScaler(), 'regressor': Lasso()}

In [ ]:
sklearn.metrics.mean_squared_error(y_true, y_pred, *, sample_weight=None, multioutput='uniform_average', squared=True)

In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler  # 전처리
from sklearn.pipeline import Pipeline # 파이프
from sklearn.model_selection import GridSearchCV # cross-validation

from sklearn.linear_model import LinearRegression, Ridge, RANSACRegressor, RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor

#from lightgbm import LGBMRegressor

pip = Pipeline((("preprocess", MinMaxScaler()), ("regressor", LinearRegression()))) # 디폴트 파이프라인 생성


#파라미터
import numpy as np
class LinearRegressionGD(object):
    """ Ordinary Linear Regression , Gradient Descent"""
    def __init__(self,eta=0.001,n_iter = 20):
        self.eta = eta
        self.n_iter = 20
        
    def fit(self,X,y):
        self.w_ = np.zeros(1+X.shape[1])
        self.cost_ = []
        
        for i in range(self.n_iter):
            output = self.net_input(X)
            errors = (y - output)
            self.w_[1:]+= self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            cost = (errors**2).sum() /2.0
            self.cost_.append(cost)
            
        return self
    
    def net_input(self,X):
        return np.dot(X,self.w_[1:]) + self.w_[0]
    
    def predict(self,X):
        return self.net_input(X)


grid_parm = [
    
    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'regressor': [LinearRegression()],
     'n_jobs': -1 },

    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'classifier': [SVC()],
     'classifier__gamma': [1,10,100],
     'classifier__kernel': ['poly', 'rbf']},

    {'preprocess': [None],
     'classifier': [GaussianNB()]},
    
 
    {'preprocess': [None],
     'classifier': [DecisionTreeClassifier()],
     'classifier__criterion': ['gini','entropy'],
     'classifier__max_depth': list(range(2,15)) },
   
    {'preprocess': [None],
     'classifier': [RandomForestClassifier()],
     'classifier__n_estimators': [50,100,150] },

    {'preprocess': [None],
     'classifier': [AdaBoostClassifier()],
     'classifier__n_estimators': [50,100,150] },
   
    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'classifier': [MLPClssifier],
     'classifier_hidden_layet_sizes': [(100,),(100,100),(100,100,100)],  # 세번째, 히든레이어 세개, 노드개수 100개
     'classifier_max_iter' :[1000]}
]



from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
estimators = [('ridge', RidgeCV()),
              ('lasso', LassoCV(random_state=42)),
               ('knr', KNeighborsRegressor(n_neighbors=20,
                                           metric='euclidean'))]

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor


estimators = [('ridge', RidgeCV()),
              ('lasso', LassoCV(random_state=42)),
               ('knr', KNeighborsRegressor(n_neighbors=20,
                                           metric='euclidean'))]


final_estimator = GradientBoostingRegressor(
     n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1,
     random_state=42)

reg = StackingRegressor(
     estimators=estimators,
     final_estimator=final_estimator)

NameError: ignored

In [ ]:
from sklearn.neighbors import KNeighborsRegressor #https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor


from sklearn import tree #tree.DecisionTreeRegressor()
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, VotingRegressor

from lightgbm import LGBMRegressor

from sklearn import svm


svm.SVR()

In [ ]:


parm_grid = [
    {'n_neighbors' : [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23]}
     #{'n_neighbors' : list(range(1,25,2))} # 리스트
]

In [43]:
from sklearn.svm import SVC
?SVC

In [ ]:
grid_parm = [
    
    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'model': [KNeighborsClassifier()],
     'classifier__n_neighbors': list(range(1,25,2))},

    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'classifier': [SVC()],
     'classifier__gamma': [1,10,100],
     'classifier__kernel': ['poly', 'rbf']},

    {'preprocess': [None],
     'classifier': [GaussianNB()]},
    
 
    {'preprocess': [None],
     'classifier': [DecisionTreeClassifier()],
     'classifier__criterion': ['gini','entropy'],
     'classifier__max_depth': list(range(2,15)) },
   
    {'preprocess': [None],
     'classifier': [RandomForestClassifier()],
     'classifier__n_estimators': [50,100,150] },

    {'preprocess': [None],
     'classifier': [AdaBoostClassifier()],
     'classifier__n_estimators': [50,100,150] },
   
    {'preprocess': [StandardScaler(), MinMaxScaler()],
     'classifier': [MLPClssifier],
     'classifier_hidden_layet_sizes': [(100,),(100,100),(100,100,100)],  # 세번째, 히든레이어 세개, 노드개수 100개
     'classifier_max_iter' :[1000]}
]


In [ ]:
reg = XGBRegressor(
            n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            max_depth=5, ## 개별 나무의 최대 깊이
            gamma = 0, ## gamma
            reg_lambda = 1, ## tuning parameter of l2 penalty
            random_state=100
        ).fit(X,y)

        

In [ ]:
>>> final_layer_rfr = RandomForestRegressor(
...     n_estimators=10, max_features=1, max_leaf_nodes=5,random_state=42)
>>> final_layer_gbr = GradientBoostingRegressor(
...     n_estimators=10, max_features=1, max_leaf_nodes=5,random_state=42)
>>> final_layer = StackingRegressor(
...     estimators=[('rf', final_layer_rfr),
...                 ('gbrt', final_layer_gbr)],
...     final_estimator=RidgeCV()
...     )
>>> multi_layer_regressor = StackingRegressor(
...     estimators=[('ridge', RidgeCV()),
...                 ('lasso', LassoCV(random_state=42)),
...                 ('knr', KNeighborsRegressor(n_neighbors=20,
...                                             metric='euclidean'))],
...     final_estimator=final_layer
... )
>>> multi_layer_regressor.fit(X_train, y_train)